In [68]:
import numpy as np
import sys
import os
import matplotlib.pyplot as plt
import copy
import torch as tr

# import scripts
from src.es import random_opt
from src.es import evolutionary_strategy
from src.grad_opt import ADAM_opt
from src.api_helper import api_utils
from src.api_helper import env

device = tr.device("cuda" if tr.cuda.is_available() else "cpu")
dtype = tr.float32

##### data

In [69]:
@api_utils.multi_process_transform
def api(query): return env.rosenbrock(query)  # computation on cpu

x0, y0 = tr.rand(2).view(1, -1), tr.tensor([[-1]], dtype=dtype).view(-1, 1)  
r0 = env.rosenbrock(x0)  # initial reward on which normalisation based

x0, y0= x0.to(device), y0.to(device)  # data in GPU

print("init query", x0); print(f"init reward {api(x0, r0, device)}, origin reward: {r0.item():.2f}")  # x,y stay on GPU while r0 in cpu

init query tensor([[0.8593, 0.9924]])


PicklingError: Can't pickle <function api at 0x7f3da6c618b0>: it's not the same object as __main__.api

##### OPT

In [3]:
# hyperparameters
T = 10  # total number of iterations

# gp; includes "MA2.5", "SE", "RQ", "LR", "PO"
gp_name, gp_params = "MA2.5",{
                          "mode": "raw",      # "raw", "add", "pro" for GPs
                          "opt":"ADAM",  # opt for MLE; (quasi_newton, ADAM)
                          "epochs":128,       # epoch to run, if chosen ADAM
                          "lr":1e-1,          # learning rate for ADAM
                         }
# q-parallelism (if use analytical acq_func, q must be 1)
batch_size = 2

acq_params = { 
    "acq_name" : "qUCB",          # acqu func; includes: "EI", "UCB", "qEI", "qUCB", "qKG"
    "N_start": 32,               # number of starts for multi-start SGA
    "raw_samples" :512,          # heuristic initialisation 
    "N_MC_sample" : 256,         # number of samples for Monte Carlo simulation
    "num_fantasies": 128,        # number of fantasies used by KG
    "beta":1.,                   # used by UCB/qUCB
               }

bayes_opt = bayesian_optimiser(gp_name, gp_params, device, acq_params)

In [4]:
%%time
xs, ys = bayes_opt.outer_loop(T, (-4,4),x0, y0, r0, api, batch_size)  # maximising reward

NLML: -3.57
Iter: 1, reward: -1,433.98
NLML: 158,658,896.00
Iter: 2, reward: -18.10
NLML: 2,119,508.00
Iter: 3, reward: -3.37
NLML: 559,796.44
Iter: 4, reward: -0.55
NLML: 150,972.84
Iter: 5, reward: -0.62
NLML: 66,763.80
Iter: 6, reward: -1.30
NLML: 37,455.06
Iter: 7, reward: -1.08
NLML: 37,780.32
Iter: 8, reward: -0.79
NLML: 26,172.46
Iter: 9, reward: -2.92
NLML: 19,244.47
Iter: 10, reward: -0.52
CPU times: user 10.4 s, sys: 20 s, total: 30.4 s
Wall time: 7.79 s
